In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
df_base_atestados = pd.read_excel("Automatização - Ambulatório.xlsx")

In [4]:
df_base_atestados.columns = ['data_sap','matricula','nome','idade','sexo','funcao','loja','data_inicial','data_final','vai_inss','dias_perdidos','cib_10','grupo_doenca','lcdo_soc','','','']

In [5]:
df_base_atestados_filtrado = df_base_atestados[['data_sap','matricula','nome','idade','sexo','funcao','loja','data_inicial','data_final','vai_inss','dias_perdidos','cib_10','grupo_doenca','lcdo_soc']].copy()

In [6]:
df_base_atestados_filtrado = df_base_atestados_filtrado[~df_base_atestados_filtrado['matricula'].isnull()].copy()

In [7]:
df_base_atestados_filtrado = df_base_atestados_filtrado[df_base_atestados_filtrado['nome'] != 'NOME'].copy()

In [8]:
df_atestados_meses = df_base_atestados_filtrado[['data_inicial']].copy()

In [9]:
### Pegando somente o dado que necessita que é o mês
df_atestados_meses['data_inicial'] = df_atestados_meses['data_inicial'].astype(str)

In [10]:
df_atestados_meses['data_inicial'] = df_atestados_meses['data_inicial'].str.replace(".","-")
df_atestados_meses['data_inicial'] = df_atestados_meses['data_inicial'].str.replace(":","-")
df_atestados_meses['data_inicial'] = df_atestados_meses['data_inicial'].str.replace(" ","-")

In [11]:
df_atestados_meses.head(2)

,data_inicial
0,2019-05-08-00-00-00
1,2019-05-10-00-00-00


In [12]:
df_atestados_meses['mes'] = df_atestados_meses['data_inicial'].apply(lambda item: str(item).split('-')[1])

In [13]:
df_atestados_meses.drop(columns = {'data_inicial'}, inplace = True)

## ------ Quantidade de atestados por mês -----

In [14]:
df_atestados_meses['mes'] = df_atestados_meses['mes'].astype(int)

In [15]:
df_atestados_meses['soma_zerada'] = df_atestados_meses['mes']   

In [20]:
df_atestados_meses = df_atestados_meses.groupby('mes', as_index = False).agg({'soma_zerada': 'count'})

In [ ]:
df_atestados_meses.rename(columns = {''})

In [22]:
df_atestados_meses.T

,0,1,2,3,4,5
mes,5,6,7,8,9,10
soma_zerada,152,162,200,199,153,199
